In [144]:
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Campero/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [145]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
#from numpy import ndarray as nda
import time
import pandas as pd
import json

In [148]:
docs_path = 'data/tweet_document_ids_map.csv'
with open(docs_path) as fp:
    lines = fp.readlines()
lines = [l.strip().replace(' +', ' ') for l in lines]
print("Total number of tweet documents in the corpus: {}".format(len(lines)))

Total number of tweet documents in the corpus: 4000


In [149]:
tweets_path = 'data/tw_hurricane_data.json'
with open(tweets_path) as fp:
    tweets = fp.readlines()
tweets = [t for t in tweets]
print("Total number of tweets in the corpus: {}".format(len(tweets)))

Total number of tweets in the corpus: 4000


In [151]:
tweets_json=[]
i=0
for t in tweets:
    if i != 4000:
        tweets_json.append(json.loads(t)) #The element 4000 is an empty string, if we don't avoid it the code breaks
    i+=1

In [152]:
dt_tweets = pd.DataFrame.from_dict(data=tweets_json)
dt_tweets.head()

created_at                   id               id_str  \
0  Fri Sep 30 18:39:08 +0000 2022  1575918182698979328  1575918182698979328   
1  Fri Sep 30 18:39:01 +0000 2022  1575918151862304768  1575918151862304768   
2  Fri Sep 30 18:38:58 +0000 2022  1575918140839673873  1575918140839673873   
3  Fri Sep 30 18:38:57 +0000 2022  1575918135009738752  1575918135009738752   
4  Fri Sep 30 18:38:53 +0000 2022  1575918119251419136  1575918119251419136   

                                           full_text  truncated  \
0  So this will keep spinning over us until 7 pm…...      False   
1  Our hearts go out to all those affected by #Hu...      False   
2  Kissimmee neighborhood off of Michigan Ave. \n...      False   
3  I have this one tree in my backyard that scare...      False   
4  @AshleyRuizWx @Stephan89441722 @lilmizzheidi @...      False   

  display_text_range                                           entities  \
0            [0, 76]  {'hashtags': [{'text': 'HurricaneIan', 'indice...   
1           [0, 136]  {'hashtags': [{'text': 'HurricaneIan', 'indice...   
2            [0, 58]  {'hashtags': [{'text': 'HurricaneIan', 'indice...   
3           [0, 141]  {'hashtags': [{'text': 'scwx', 'indices': [122...   
4         [127, 280]  {'hashtags': [{'text': 'HurricaneIan', 'indice...   

                                   extended_entities  \
0  {'media': [{'id': 1575918178261254162, 'id_str...   
1                                                NaN   
2  {'media': [{'id': 1575918121080311808, 'id_str...   
3                                                NaN   
4                                                NaN   

                                            metadata  \
0  {'iso_language_code': 'en', 'result_type': 're...   
1  {'iso_language_code': 'en', 'result_type': 're...   
2  {'iso_language_code': 'en', 'result_type': 're...   
3  {'iso_language_code': 'en', 'result_type': 're...   
4  {'iso_language_code': 'en', 'result_type': 're...   

                                              source  ...  is_quote_status  \
0  <a href="http://twitter.com/download/iphone" r...  ...            False   
1  <a href="https://sproutsocial.com" rel="nofoll...  ...            False   
2  <a href="http://twitter.com/download/iphone" r...  ...            False   
3  <a href="http://twitter.com/download/iphone" r...  ...            False   
4  <a href="http://twitter.com/download/iphone" r...  ...            False   

  retweet_count  favorite_count favorited retweeted possibly_sensitive lang  \
0             0               0     False     False              False   en   
1             0               0     False     False                NaN   en   
2             0               0     False     False              False   en   
3             0               0     False     False                NaN   en   
4             0               0     False     False                NaN   en   

  quoted_status_id quoted_status_id_str quoted_status  
0              NaN                  NaN           NaN  
1              NaN                  NaN           NaN  
2              NaN                  NaN           NaN  
3              NaN                  NaN           NaN  
4              NaN                  NaN           NaN  

[5 rows x 30 columns]

In [153]:
print(dt_tweets.iloc[2]['full_text'])

Kissimmee neighborhood off of Michigan Ave. 
#HurricaneIan https://t.co/jf7zseg0Fe


In [154]:
print(dt_tweets.iloc[510]['entities']['urls'])

[{'url': 'https://t.co/RqBeVaqVr9', 'expanded_url': 'https://washingtoncurrent.substack.com/p/ians-horrific-impact-once-again-throws?sd=pf', 'display_url': 'washingtoncurrent.substack.com/p/ians-horrifi…', 'indices': [193, 216]}]


## HINT
Take into account that for future queries, the final output must return (when
present) the following information for each of the selected documents: Tweet |
Username | Date | Hashtags | Likes | Retweets | Url (here the “Url” means the
tweet link).

## Marta: el url no he sapigut detectar quin era així q de moment ho he deixat buit... I els hastags no estic del tot segura el q volem...

In [155]:
print(list(dt_tweets.columns))

['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'extended_entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status']


In [156]:
tweets_query_format = dt_tweets[['full_text', 'user', 'created_at', 'favorite_count', 'retweet_count']]
username = [x['name'] for x in list(dt_tweets['user'])]
hashtags = [  [ht['text'] for ht in x['hashtags']]  for x in list(dt_tweets['entities'])]
urls =[]
for i in range(len(dt_tweets)):
    url = dt_tweets.iloc[i]['entities']['urls'][0]['url'] if dt_tweets.iloc[i]['entities']['urls'] and len(dt_tweets.iloc[i]['entities']['urls'])>0 else None
    urls.append(url)
print(len(urls))

4000


In [158]:
tweets_query_format['user'] = username
tweets_query_format['hashtags'] = hashtags
tweets_query_format['url'] = urls

In [159]:
tweets_query_format.tail()

full_text                  user  \
3995  The CFRD, @CarrboroPD , Carrboro Public Works,...  Carrboro Fire-Rescue   
3996  Why isn’t @OsceolaCountyFl listed on the @fema...         BaconBitsNews   
3997  So it really wasn't #HurricaneIan that flooded...              @jganyfl   
3998  Damage in my area in Punta Gorda...a thread. I...             CJ Haddad   
3999  This is awful.   https://t.co/BzhXaAPv7B￼\n\n#...      Ohemgeeitsalyssa   

                          created_at  favorite_count  retweet_count  \
3995  Fri Sep 30 14:33:06 +0000 2022               2              0   
3996  Fri Sep 30 14:33:01 +0000 2022               0              0   
3997  Fri Sep 30 14:32:57 +0000 2022              16              8   
3998  Fri Sep 30 14:32:56 +0000 2022               2              1   
3999  Fri Sep 30 14:32:56 +0000 2022               0              0   

                                               hashtags  \
3995                 [CarrboroSafe, ncwx, HurricaneIan]   
3996  [Kissimmee, SaintCloud, BlueCounty, Disney, De...   
3997                 [HurricaneIan, Florida, MAGATears]   
3998                                     [HurricaneIan]   
3999                           [DeSantis, HurricaneIan]   

                          url  
3995  https://t.co/jrmrS3tJXa  
3996  https://t.co/JaOkK6skP9  
3997                     None  
3998                     None  
3999  https://t.co/BzhXaAPv7B

In [160]:
tweets_query_format.head()

full_text               user  \
0  So this will keep spinning over us until 7 pm…...               Suz👻   
1  Our hearts go out to all those affected by #Hu...               Lytx   
2  Kissimmee neighborhood off of Michigan Ave. \n...  Christopher Heath   
3  I have this one tree in my backyard that scare...             alex ✨   
4  @AshleyRuizWx @Stephan89441722 @lilmizzheidi @...             Tess 💋   

                       created_at  favorite_count  retweet_count  \
0  Fri Sep 30 18:39:08 +0000 2022               0              0   
1  Fri Sep 30 18:39:01 +0000 2022               0              0   
2  Fri Sep 30 18:38:58 +0000 2022               0              0   
3  Fri Sep 30 18:38:57 +0000 2022               0              0   
4  Fri Sep 30 18:38:53 +0000 2022               0              0   

               hashtags   url  
0        [HurricaneIan]  None  
1        [HurricaneIan]  None  
2        [HurricaneIan]  None  
3  [scwx, HurricaneIan]  None  
4        [HurricaneIan]  None

In [162]:
list(tweets_query_format['url']).count(None)

2412

### OJO: hay algo mal porque mas de la mitad de tweets (2412) no tienen su url...

In [204]:
def build_terms(line):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
    line= line.lower() ## Transform in lowercase
    line=line.replace('@', '')
    line=line.replace('#', '')
    line=line.replace('.', '')
    line= line.split() ## Tokenize the text to get a list of terms
    line=[x for x in line if x not in stop_words]  ##eliminate the stopwords (HINT: use List Comprehension)
    line = filter(lambda x:x[0:5]!='https', line)
    line=[stemmer.stem(x) for x in line] ## perform stemming (HINT: use List Comprehension)
    ## END CODE
    return line

In [201]:
ss= "patata https://t.co/bzhxaapv7b"
print(" ".join(filter(lambda x:x[0:5]!='https', ss.split())))

patata


In [205]:
tweets_query_format_processed=pd.DataFrame(columns=['Tweet', 'Username', 'Date', 'Hashtags', 'Likes', 'Retweets', 'Url'])
tweets_query_format_processed = tweets_query_format.copy()
for index, row in tweets_query_format_processed.iterrows():
    tweets_query_format_processed.iloc[index]['Tweet']=build_terms(row['Tweet'])
print(tweets_query_format_processed.head())

                                               Tweet              Username  \
0  [aw, desanti, busi, scheme, traffick, asylum, ...      Ohemgeeitsalyssa   
1  [damag, area, punta, gordaa, thread, tropic, g...             CJ Haddad   
2  [realli, hurricaneian, flood, florida, magatea...              @jganyfl   
3  [isn’t, osceolacountyfl, list, fema, website?,...         BaconBitsNews   
4  [cfrd,, carrboropd, ,, carrboro, public, works...  Carrboro Fire-Rescue   

                             Date      Hashtags Likes Retweets   Url  
0  Fri Sep 30 14:32:56 +0000 2022      DeSantis     0        0  None  
1  Fri Sep 30 14:32:56 +0000 2022  HurricaneIan     2        1  None  
2  Fri Sep 30 14:32:57 +0000 2022  HurricaneIan    16        8  None  
3  Fri Sep 30 14:33:01 +0000 2022     Kissimmee     0        0  None  
4  Fri Sep 30 14:33:06 +0000 2022  CarrboroSafe     2        0  None  


In [18]:
print(build_terms("One day I saw a cow without a tie dressed with a uniform"))

['one', 'day', 'saw', 'cow', 'without', 'tie', 'dress', 'uniform']


In [81]:
def create_index(lines):
    """
    Implement the inverted index
    
    Argument:
    lines -- collection of Wikipedia articles
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of documents where these keys appears in (and the positions) as values.
    """
    index = defaultdict(list)
    title_index = {}  # dictionary to map page titles to page ids
    for line in lines:  # Remember, lines contain all documents: article-id | article-title | article-body
        #line_arr = line.split("\")
        #print(line.split("\t"))
        line_arr=line.split("\t")[1]
        page_id = int(line_arr)
        terms = build_terms(''.join(line_arr[1:])) # page_title + page_text
        title = line_arr[1]
        title_index[page_id]=title  ## we do not need to apply get terms to title because it used only to print titles and not in the index
        
        ## ===============================================================        
        ## create the index for the current page and store it in current_page_index (current_page_index)
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,3]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list
                
        ## START CODE
                current_page_index[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[page_id, array('I',[position])] #'I' indicates unsigned int (int in Python)
            
        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
        
        ## END CODE                    
                    
    return index, title_index

In [83]:
start_time = time.time()
index, title_index = create_index(lines)
print("Total time to create the index: {} seconds".format(np.round(time.time() - start_time, 2)))

Total time to create the index: 3.61 seconds
